<a href="https://colab.research.google.com/github/YBen1/Analyse_Flight_Data/blob/main/DayTradeTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I try boild a prediction based on the beginning of the day

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

In [2]:
!pip install yfinance

     |████████████████████████████████| 63 kB 657 kB/s 
     |████████████████████████████████| 6.4 MB 6.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import yfinance as yf

# Set the start and end date
start_date = '2021-01-01'
end_date = '2022-01-01'

# Set the ticker
ticker = 'AMZN'

# Get the data
data = yf.download(ticker, start_date, end_date, interval = "1h")

# Print 5 rows
data.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
2021-12-31 11:30:00-05:00,3363.379883,3364.500000,3351.000000,3352.729980,3352.729980,230742
2021-12-31 12:30:00-05:00,3352.290039,3359.379883,3350.175293,3354.300049,3354.300049,175627
2021-12-31 13:30:00-05:00,3353.159912,3353.449951,3345.250000,3348.330078,3348.330078,262655
2021-12-31 14:30:00-05:00,3348.879883,3350.879883,3340.000000,3342.500000,3342.500000,236082
2021-12-31 15:30:00-05:00,3342.889893,3346.000000,3331.169922,3334.719971,3334.719971,500351


Lets get only the openings

In [43]:
import datetime
stock_opening = data.loc[data.index.hour == 9]

In [44]:
stock_day = data.loc[data.index.hour!= 9]
min_and_max = stock_day.groupby(stock_day.index.date).agg({'High': ['max'], 'Low' : ['min']})

In [46]:
y = stock_day.groupby(stock_day.index.date).agg({'High': ['max']})
feature_columns = ['Open', 'High','Low', 'Close' ,'Adj Close', 'Volume']
X = stock_opening[feature_columns]

In [49]:
stock_model= DecisionTreeRegressor()
# Fit Model
stock_model.fit(X, y)

DecisionTreeRegressor()

In [50]:
train_X, val_X, train_y, val_y = train_test_split(X,y, random_state = 1)

from sklearn.tree import DecisionTreeRegressor

stock_model = DecisionTreeRegressor(random_state=1)

# Fit iowa_model with the training data.
stock_model.fit(train_X, train_y)

DecisionTreeRegressor(random_state=1)

In [51]:
val_predictions = stock_model.predict(val_X)

In [52]:
print(val_predictions[:5])
# print the top few actual prices from validation data
print(val_y.head())

[3385.         3334.35009766 3482.66992188 3314.51000977 3223.37988281]
                   High
                    max
2021-04-12  3393.000000
2021-12-31  3365.859619
2021-12-02  3475.600098
2021-08-24  3315.494141
2021-05-14  3228.860107


In [53]:
from sklearn.metrics import mean_absolute_error
val_mae = mean_absolute_error(val_y, val_predictions)

In [54]:
val_mae

22.788922991071427